In [2]:
from pyqtgraph.Qt import QtCore, QtGui; from pyqtgraph.ptime import time
import socket, re, random, numpy as np, pyqtgraph as pg

UDP_PORT, UDP_IP = 8080, "192.168.43.14"
w_size = 100

sock = socket.socket( socket.AF_INET, socket.SOCK_DGRAM )
sock.bind( (UDP_IP,UDP_PORT) )

app = QtGui.QApplication([])
win = pg.GraphicsWindow()
win.setWindowTitle('BMI160')
pg.setConfigOptions(antialias=True)

all_data = sock.recvfrom( 2500 )[0]
parts = re.split(',', all_data)
n_ofdata = len(parts)

#n_ofdata = 3
axis_nms = ["Gx", "Gy", "Gz", "Ax", "Ay", "Az"]

if (n_ofdata != len(axis_nms)):
    axis_nms = []
    print "Всего найдено %d потоков данных" %n_ofdata
    for i in range(n_ofdata):
        input_text = 'Enter file name for %d axis: ' %i
        answer = raw_input(input_text)
        axis_nms.append(answer)

p = [None] * n_ofdata
data = np.zeros([n_ofdata, w_size])
curves = [None] * n_ofdata
ptr = np.zeros(n_ofdata)

for i in range(n_ofdata):
    p[i] = win.addPlot(title = axis_nms[i])
    curves[i] = p[i].plot(data[i], pen = (random.randint(0, 255)*10*i%255, random.randint(150, 255), random.randint(50, 255)))
    
    if ((i + 1) % 3 == 0) and (i > 0):
        win.nextRow()
    else:
        pass

def updater(n_axis):
    global data, curves, ptr
    all_data = sock.recvfrom( 214748364 )[0]
    parts = re.split(',', all_data)
    data[n_axis][:-1] = data[n_axis][1:]
    axis_data = int(parts[n_axis])
    if (n_axis < 3):
        axis_data = axis_data * 500.0 / 32768.0
    elif (n_axis < 6):
        axis_data = axis_data * 8.0 / 32768.0
    else:
        pass
    data[n_axis][-1] = axis_data
    curves[n_axis].setData(data[n_axis])
    ptr[n_axis] += 1
    
def full_updater():    
    for i in range(n_ofdata):
        updater(i)

timer = pg.QtCore.QTimer()
timer.timeout.connect(full_updater)
timer.start(0)

if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

'\nfrom pyqtgraph.Qt import QtCore, QtGui; from pyqtgraph.ptime import time\nimport socket, re, random, numpy as np, pyqtgraph as pg\n\nUDP_PORT, UDP_IP = 8080, "192.168.43.14"\nw_size = 100\n\nsock = socket.socket( socket.AF_INET, socket.SOCK_DGRAM )\nsock.bind( (UDP_IP,UDP_PORT) )\n\napp = QtGui.QApplication([])\nwin = pg.GraphicsWindow()\nwin.setWindowTitle(\'BMI160\')\npg.setConfigOptions(antialias=True)\n\nall_data = sock.recvfrom( 2500 )[0]\nparts = re.split(\',\', all_data)\nn_ofdata = len(parts)\n\n#n_ofdata = 3\naxis_nms = ["Gx", "Gy", "Gz", "Ax", "Ay", "Az"]\n\nif (n_ofdata != len(axis_nms)):\n    axis_nms = []\n    print "\xd0\x92\xd1\x81\xd0\xb5\xd0\xb3\xd0\xbe \xd0\xbd\xd0\xb0\xd0\xb9\xd0\xb4\xd0\xb5\xd0\xbd\xd0\xbe %d \xd0\xbf\xd0\xbe\xd1\x82\xd0\xbe\xd0\xba\xd0\xbe\xd0\xb2 \xd0\xb4\xd0\xb0\xd0\xbd\xd0\xbd\xd1\x8b\xd1\x85" %n_ofdata\n    for i in range(n_ofdata):\n        input_text = \'Enter file name for %d axis: \' %i\n        answer = raw_input(input_text)\n        ax